In [1]:
import geopandas as gpd
import pandas as pd
from shapely.geometry import Point
%matplotlib inline
import seaborn as sns; sns.set_theme(color_codes=True)
pd.set_option('display.max_columns', None)
import matplotlib.pyplot as plt
import numpy as np
import matplotlib.colors
import warnings
warnings.filterwarnings('ignore')
from scipy import stats
import math    
import folium
from scipy.cluster.hierarchy import linkage, fcluster
from scipy.spatial.distance import pdist
import sys
from haversine import haversine
from shapely.geometry import LineString
from shapely.ops import split, linemerge
import krippendorff

In [2]:
import sys
sys.path.append('../')

from src.cluster import *
from src.colors import *
from src.map import *
from src.compare import *

In [4]:
#read in seattle validation labels
seattle_labels = gpd.read_file('../data/round-3/seattle-validation-labels-round3-230927.geojson')
seattle_labels = seattle_labels.drop(columns=['time_created'])
seattle_labels.head()

,username,label_id,street_edge_id,neighborhood,label_type,lat,lng,gsv_panorama_id,correct,severity,temporary,tag_list,description,geometry
0,Devon Snyder,8263,519,Greenwood,Obstacle,47.697910,-122.346878,eA8O01ruC7m2Hxna2rOYmA,None,5,t,sign,None,POINT (-122.34688 47.69791)
1,Devon Snyder,8264,519,Greenwood,Obstacle,47.697910,-122.346840,eA8O01ruC7m2Hxna2rOYmA,None,5,t,sign,None,POINT (-122.34684 47.69791)
2,Devon Snyder,8265,519,Greenwood,SurfaceProblem,47.697731,-122.347191,u54hhkI1_JMvO6PSVuls_Q,None,1,f,cracks,None,POINT (-122.34719 47.69773)
3,Devon Snyder,8266,519,Greenwood,Obstacle,47.697914,-122.346817,-bwvo2vUzCb2YOsT0bKDGQ,None,5,t,sign,None,POINT (-122.34682 47.69791)
4,Devon Snyder,8267,519,Greenwood,SurfaceProblem,47.697880,-122.346687,-bwvo2vUzCb2YOsT0bKDGQ,None,1,f,utility panel,None,POINT (-122.34669 47.69788)


In [6]:
#split Mikey and Devon's labels
mikey = seattle_labels[seattle_labels['username'] == 'mikey']
devon = seattle_labels[seattle_labels['username'] == 'Devon Snyder']


In [34]:
def cluster_high_severity_with_the_other(df1,df2,severity_threshold,label_type_id):
    #select only Mikey's labels > 3 for severity
    df1 = df1[df1['severity'] >= severity_threshold]
    # concact with df2
    df = pd.concat([df1,df2])
    label_type_clusters = cluster_label_type_at_index(df,label_type_id)[2]
    label_type_clusters['clustered'] = label_type_clusters.duplicated(subset=['cluster_id']).astype(int)
    #count clustered == 1 and divide by len(df1) 
    print(str(label_types[label_type_id])+ '\n'  + str(label_type_clusters['clustered'].value_counts()))
    print("clustered percentage for df1: " + str(label_type_clusters['clustered'].value_counts()[1]/len(df1)))
    print("clustered percentage for df2: " + str(label_type_clusters['clustered'].value_counts()[1]/len(df2))) 

In [38]:
cluster_high_severity_with_the_other (mikey,devon,3,5)

Obstacle
0    45
1    10
Name: clustered, dtype: int64
clustered percentage for df1: 0.17543859649122806
clustered percentage for df2: 0.023696682464454975


In [41]:
cluster_high_severity_with_the_other (devon,mikey,3,5)

Obstacle
0    49
1     9
Name: clustered, dtype: int64
clustered percentage for df1: 0.1836734693877551
clustered percentage for df2: 0.023316062176165803
